In [3]:
!pip install azure-storage-blob
!pip install pyarrow
!pip install psycopg2 sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.3 MB/s eta 0:00:00


In [9]:
!pip install SQLAlchemy


In [4]:
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar

In [24]:
URL = 'https://data.cdc.gov/api/views/9j2v-jamp/rows.csv?accessType=DOWNLOAD'

df_raw = pd.read_csv(URL)
df_raw.head()
print(df_raw.info())
print(df_raw.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6390 entries, 0 to 6389
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   INDICATOR       6390 non-null   object 
 1   UNIT            6390 non-null   object 
 2   UNIT_NUM        6390 non-null   int64  
 3   STUB_NAME       6390 non-null   object 
 4   STUB_NAME_NUM   6390 non-null   int64  
 5   STUB_LABEL      6390 non-null   object 
 6   STUB_LABEL_NUM  6390 non-null   float64
 7   YEAR            6390 non-null   int64  
 8   YEAR_NUM        6390 non-null   int64  
 9   AGE             6390 non-null   object 
 10  AGE_NUM         6390 non-null   float64
 11  ESTIMATE        5484 non-null   float64
 12  FLAG            906 non-null    object 
dtypes: float64(3), int64(4), object(6)
memory usage: 649.1+ KB
None
(6390, 13)


In [29]:
import json
from io import StringIO
from azure.storage.blob import BlobServiceClient

config = {
    "connectionString": "DefaultEndpointsProtocol=https;AccountName=stcis4400springev;AccountKey=ueJ5MFF2ZkPk0k1xgDLDEpJH4TyfhzgpoXyQwOFYfcoW5Cwtxmxvh+DTT7EMzFlvMCIv0N1C6bN1+AStug/Q+A==;EndpointSuffix=core.windows.net"
}

config_file_path = 'config.json'

with open(config_file_path, 'w') as config_file:
    json.dump(config, config_file)

CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = 'deathratetwo'
blob_name = "deathratetwo.csv"

output = StringIO()
df_raw.to_csv(output, index=False)
data = output.getvalue()
output.close()

blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

blob_client.upload_blob(data, overwrite=True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")


Uploaded deathratetwo.csv to Azure Blob Storage in container deathratetwo.


config.json = {
    
    "connectionString": "DefaultEndpointsProtocol=https;AccountName=stcis4400springev;AccountKey=ueJ5MFF2ZkPk0k1xgDLDEpJH4TyfhzgpoXyQwOFYfcoW5Cwtxmxvh+DTT7EMzFlvMCIv0N1C6bN1+AStug/Q+A==;EndpointSuffix=core.windows.net"
}

In [30]:
config_file_path = 'config.json'

with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = 'deathratetwo'

blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

container_client = blob_service_client.get_container_client(CONTAINER_AZURE)

death_rate = pd.DataFrame()

blob_list = container_client.list_blobs()
for blob in blob_list:
    print(blob.name)
    blob_client = container_client.get_blob_client(blob=blob.name)
    blob_data = blob_client.download_blob()
    blob_content = blob_data.readall().decode('utf-8')
    df = pd.read_csv(StringIO(blob_content))
    print(df.shape)
    death_rate_df = df.copy()


deathratetwo.csv
(6390, 13)


In [31]:
death_rate_df.columns


Index(['INDICATOR', 'UNIT', 'UNIT_NUM', 'STUB_NAME', 'STUB_NAME_NUM',
       'STUB_LABEL', 'STUB_LABEL_NUM', 'YEAR', 'YEAR_NUM', 'AGE', 'AGE_NUM',
       'ESTIMATE', 'FLAG'],
      dtype='object')

In [32]:
death_rate_df.head()

,INDICATOR,UNIT,UNIT_NUM,STUB_NAME,STUB_NAME_NUM,STUB_LABEL,STUB_LABEL_NUM,YEAR,YEAR_NUM,AGE,AGE_NUM,ESTIMATE,FLAG
0,Death rates for suicide,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.0,1950,1,All ages,0.0,13.2,NaN
1,Death rates for suicide,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.0,1960,2,All ages,0.0,12.5,NaN
2,Death rates for suicide,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.0,1970,3,All ages,0.0,13.1,NaN
3,Death rates for suicide,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.0,1980,4,All ages,0.0,12.2,NaN
4,Death rates for suicide,"Deaths per 100,000 resident population, age-ad...",1,Total,0,All persons,0.0,1981,5,All ages,0.0,12.3,NaN


In [33]:
columns_to_drop = ['UNIT', 'UNIT_NUM', 'STUB_NAME', 'STUB_NAME_NUM', 'STUB_LABEL', 'STUB_LABEL_NUM', 'FLAG']
cleaned_df = death_rate_df.drop(columns=columns_to_drop)

cleaned_df['YEAR'] = cleaned_df['YEAR'].astype(int)
cleaned_df['AGE'] = cleaned_df['AGE'].astype(str)

cleaned_df.rename(columns={'INDICATOR': 'Indicator', 'YEAR': 'Year', 'AGE': 'Age', 'ESTIMATE': 'Death_rate'}, inplace=True)

cleaned_df = cleaned_df.dropna(subset=['Death_rate'])

cleaned_df.reset_index(drop=True, inplace=True)

print(cleaned_df.head())


                 Indicator  Year  YEAR_NUM       Age  AGE_NUM  Death_rate
0  Death rates for suicide  1950         1  All ages      0.0        13.2
1  Death rates for suicide  1960         2  All ages      0.0        12.5
2  Death rates for suicide  1970         3  All ages      0.0        13.1
3  Death rates for suicide  1980         4  All ages      0.0        12.2
4  Death rates for suicide  1981         5  All ages      0.0        12.3


In [34]:
import pandas as pd



age_groups = {
    '0-14': ['0', '1', '2', '3', '4'],
    '15-24': ['5', '6', '7', '8', '9'],
    '25-44': ['10', '11', '12', '13'],
    '45-64': ['14', '15', '16', '17'],
    '65+': ['18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100']
}

def map_age_to_group(age):
    for group, age_range in age_groups.items():
        if age in age_range:
            return group
    return 'Unknown'

death_rate_df['Age_Group'] = death_rate_df['AGE_NUM'].apply(map_age_to_group)

print(death_rate_df.head())


                 INDICATOR                                               UNIT  \
0  Death rates for suicide  Deaths per 100,000 resident population, age-ad...   
1  Death rates for suicide  Deaths per 100,000 resident population, age-ad...   
2  Death rates for suicide  Deaths per 100,000 resident population, age-ad...   
3  Death rates for suicide  Deaths per 100,000 resident population, age-ad...   
4  Death rates for suicide  Deaths per 100,000 resident population, age-ad...   

   UNIT_NUM STUB_NAME  STUB_NAME_NUM   STUB_LABEL  STUB_LABEL_NUM  YEAR  \
0         1     Total              0  All persons             0.0  1950   
1         1     Total              0  All persons             0.0  1960   
2         1     Total              0  All persons             0.0  1970   
3         1     Total              0  All persons             0.0  1980   
4         1     Total              0  All persons             0.0  1981   

   YEAR_NUM       AGE  AGE_NUM  ESTIMATE FLAG Age_Group  
0   

In [35]:
def map_year_to_group(year):
    if year < 1980:
        return 'Before 1980'
    elif 1980 <= year < 2000:
        return '1980-1999'
    elif 2000 <= year < 2020:
        return '2000-2019'
    else:
        return '2020 and later'

cleaned_df['Year_Group'] = cleaned_df['YEAR_NUM'].apply(map_year_to_group)

print(cleaned_df.head())


                 Indicator  Year  YEAR_NUM       Age  AGE_NUM  Death_rate  \
0  Death rates for suicide  1950         1  All ages      0.0        13.2   
1  Death rates for suicide  1960         2  All ages      0.0        12.5   
2  Death rates for suicide  1970         3  All ages      0.0        13.1   
3  Death rates for suicide  1980         4  All ages      0.0        12.2   
4  Death rates for suicide  1981         5  All ages      0.0        12.3   

    Year_Group  
0  Before 1980  
1  Before 1980  
2  Before 1980  
3  Before 1980  
4  Before 1980  


In [19]:
print(cleaned_df.columns)


Index(['Indicator', 'Year', 'YEAR_NUM', 'Age', 'AGE_NUM', 'Death_rate',
       'Suicide Rate'],
      dtype='object')


In [25]:
def map_indicator_to_group(indicator):
    if 'suicide' in indicator.lower():
        return 'Suicide'
    else:
        return 'Other'

cleaned_df['Indicator_Group'] = cleaned_df['Indicator'].apply(map_indicator_to_group)

print(cleaned_df.head())



                 Indicator  Year  YEAR_NUM       Age  AGE_NUM  Death_rate  \
0  Death rates for suicide  1950         1  All ages      0.0        13.2   
1  Death rates for suicide  1960         2  All ages      0.0        12.5   
2  Death rates for suicide  1970         3  All ages      0.0        13.1   
3  Death rates for suicide  1980         4  All ages      0.0        12.2   
4  Death rates for suicide  1981         5  All ages      0.0        12.3   

  Suicide Rate   Year_Group Indicator_Group  
0         High  Before 1980         Suicide  
1         High  Before 1980         Suicide  
2         High  Before 1980         Suicide  
3         High  Before 1980         Suicide  
4         High  Before 1980         Suicide  


In [36]:
def week_of_month(dt):
  year = dt.year
  month = dt. month
  day = dt.day

  cal = calendar.monthcalendar(year, month)
  week_number = (day - 1) // 7 + 1
  return week_number

start_date = pd.to_datetine ('2024-01-01')
end_late = pd.to_datetine('2024-02-01')

date_dimension = pd.Dataframe({'date': pd.date_range(start_date, end_date, freq='H')})

date_dimension.head(25)

date_dimension['year _number'] = date_dinension["date"].dt.year
date_dinension['quarter_number'] = date_dimension ['date'].dt.quarter
date_dimension['month_number'] = date_dimension['date'].dt.month
date_dimension['monthName'] = date_dimension['date'].dt.strftine("%B")
date_dimension['daynumber'] = date_dinension['date'].dt.day
date_dimension['dayName'] = date_dimension['date'].dt.strftine('%A')
date_dimension['hour_number'] = date_dimension["date"].dt.hour
date_dimension['date_iso_format'] = date_dimension['date'].apply(lambda x: x.isoformat())
date_dimension['date_id'] = date_dimension['date'].dt.strftine ('%Y%m%d%H')

date_dimension['weekofMonth'] = date_dimension["date"].apply(week_of_month)
date_dimension['weekofYear'] = date_dimension ['date'].dt.strftine("%U")

AttributeError: module 'pandas' has no attribute 'to_datetine'

In [37]:
import pandas as pd
import calendar

def week_of_month(dt):
    year = dt.year
    month = dt.month
    day = dt.day

    cal = calendar.monthcalendar(year, month)
    week_number = (day - 1) // 7 + 1
    return week_number

start_date = pd.to_datetime('2024-01-01')
end_date = pd.to_datetime('2024-02-01')

date_dimension = pd.DataFrame({'date': pd.date_range(start_date, end_date, freq='H')})

date_dimension['year_number'] = date_dimension["date"].dt.year
date_dimension['quarter_number'] = date_dimension["date"].dt.quarter
date_dimension['month_number'] = date_dimension["date"].dt.month
date_dimension['monthName'] = date_dimension["date"].dt.strftime("%B")
date_dimension['day_number'] = date_dimension["date"].dt.day
date_dimension['dayName'] = date_dimension["date"].dt.strftime('%A')
date_dimension['hour_number'] = date_dimension["date"].dt.hour
date_dimension['date_iso_format'] = date_dimension["date"].apply(lambda x: x.isoformat())
date_dimension['date_id'] = date_dimension["date"].dt.strftime('%Y%m%d%H')

date_dimension['week_of_month'] = date_dimension["date"].apply(week_of_month)
date_dimension['week_of_year'] = date_dimension["date"].dt.strftime("%U")

new_order = ['date_id', 'date_iso_format', 'year_number', 'quarter_number', 'month_number', 'day_number', 'hour_number', 'monthName', 'dayName', 'week_of_year', 'week_of_month']
date_dimension = date_dimension[new_order]
print(date_dimension.head(25))


       date_id      date_iso_format  year_number  quarter_number  \
0   2024010100  2024-01-01T00:00:00         2024               1   
1   2024010101  2024-01-01T01:00:00         2024               1   
2   2024010102  2024-01-01T02:00:00         2024               1   
3   2024010103  2024-01-01T03:00:00         2024               1   
4   2024010104  2024-01-01T04:00:00         2024               1   
5   2024010105  2024-01-01T05:00:00         2024               1   
6   2024010106  2024-01-01T06:00:00         2024               1   
7   2024010107  2024-01-01T07:00:00         2024               1   
8   2024010108  2024-01-01T08:00:00         2024               1   
9   2024010109  2024-01-01T09:00:00         2024               1   
10  2024010110  2024-01-01T10:00:00         2024               1   
11  2024010111  2024-01-01T11:00:00         2024               1   
12  2024010112  2024-01-01T12:00:00         2024               1   
13  2024010113  2024-01-01T13:00:00         2024

In [38]:
import pandas as pd


column_mapping = {
    'Indicator': 'Indicator_Name',
    'Year': 'Year',
    'YEAR_NUM': 'Year_Number',
    'Age': 'Age_Group',
    'AGE_NUM': 'Age_Group_Number',
    'Death_rate': 'Death_Rate',
    'Suicide Rate': 'Suicide_Rate'
}

death_rate_df = death_rate_df.rename(columns=column_mapping)

print(death_rate_df.head())


                 INDICATOR                                               UNIT  \
0  Death rates for suicide  Deaths per 100,000 resident population, age-ad...   
1  Death rates for suicide  Deaths per 100,000 resident population, age-ad...   
2  Death rates for suicide  Deaths per 100,000 resident population, age-ad...   
3  Death rates for suicide  Deaths per 100,000 resident population, age-ad...   
4  Death rates for suicide  Deaths per 100,000 resident population, age-ad...   

   UNIT_NUM STUB_NAME  STUB_NAME_NUM   STUB_LABEL  STUB_LABEL_NUM  YEAR  \
0         1     Total              0  All persons             0.0  1950   
1         1     Total              0  All persons             0.0  1960   
2         1     Total              0  All persons             0.0  1970   
3         1     Total              0  All persons             0.0  1980   
4         1     Total              0  All persons             0.0  1981   

   Year_Number       AGE  Age_Group_Number  ESTIMATE FLAG Age_

In [1]:
#Database connection parameters
##server =
#database =

#password =
#driver =

#connection_string = f'DRIVER{{{driver}}};SERVER={server};DATABASE={database};UID={username};PWD={password}

#conn = pyodbc.connect(connection_string)
#cursor = conn.curson()

#cursor.setinoutsizes([(pyodbc.SQL_WVARCHAR, 0,0)])

In [39]:
death_rate_df.to_csv("deathrate.csv", index=False)

In [2]:
#Generic Functions

#def create_string(length):
# if isintance(length, int) and length > 0:
#   result_string = "(" + "?," * (length - 1) + "?)"
#   return result_string

#def insert_data(table_name, df):
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()
# result = create_string(len(df.columns))

#insert_query = f"INSERT INTO {table_name} VALUES {result}"
#print(insert_query)
#cursor.executemany(insert_query, df.values.tolist())
#conn.commit()
#conn.close()

In [13]:
from sqlalchemy import create_engine

username = 's.kataria'
pwd = 'Iamabakki1!'
hostname = 'deathrate.postgres.database.azure.com'
port = '5432'
database = 'postgres'

database_url = f'postgresql://{username}:{pwd}@{hostname}:{port}/{database}'

engine = create_engine(database_url)
